In [ ]:
# DATA STANDARDIZATION

# import libraries
import os
import shutil
from collections import defaultdict 

# Directories
input_folder = r'C:\Users\Chappyyyyyy\Documents\size_folder'
output_folder_train = r'C:\Users\Public\PartIIB project 2023_2024\Image collection without reaction\00AgNO3_mole_fraction\dataSelectorHelical5\train'
output_folder_test_val = r'C:\Users\Public\PartIIB project 2023_2024\Image collection without reaction\00AgNO3_mole_fraction\dataSelectorHelical5\testVal'
output_folder_test_val_disc = r'C:\Users\Public\PartIIB project 2023_2024\Image collection without reaction\00AgNO3_mole_fraction\dataSelectorHelical5\testValDisc'

# Self-define parameters

# number of image in input
input_num = 4

# number of size bins
num_bins = 68

# disrgardFactor x x_avg (average number of contours per bin) = the upper limit such that any bin with lower than this number is discarded
disregardFactor = 0.5

# propotion of data that will be used for traning given disrgardFactor x x_avg < x < x_avg x upperFactor / train_proportion
trainProportion = 0.8

# x_avg x upperFactor / train_proportion = below this number take 80% to train and above this number take x_avg x upperFactor to train
upperFactor = 2

# the size range of each bin
bin_width = 0.5


def obtainSizeRange(targetDir, input_num, num_bins):
    lstDir = os.listdir(targetDir)    
    numLst = [i/2 for i in range(num_bins)]
    sizeDict = {}
    for j in numLst:
      sizeDict[j] = 0
    for i in lstDir:
        t_number = int(i.split("t-")[1].split("_")[0])
        lowerT = 200 - (200/input_num) + 1
        if lowerT <= t_number <= 200:
            size = float(i[-17:-5])
            sizeRange = int(size*2)/2
            sizeDict[sizeRange] += 1

    # print(sizeDict)
    print(dict(sorted(sizeDict.items())))
    print(dict(sorted(sizeDict.items(), key=lambda item: item[1])))
    return dict(sorted(sizeDict.items()))

sizeDict = obtainSizeRange(targetDir = input_folder, input_num= input_num, num_bins= num_bins)
# [romt(sizeDict)]
totalItems = sum(sizeDict.values())
print(totalItems)

def obtainDesiredDict(sizeDict, disregardFactor, trainProportion, upperFactor, lower_limit=0, upper_limit=40):
    # get avg
    numLst = [i/2 for i in range(len(sizeDict)+1)]

    counter = 0
    for j in numLst:
        try:
            counter += sizeDict[j]
        except KeyError:
            pass    
    x_avg = counter/(len(numLst))
    
    print(f'x_avg = {x_avg}')
    
    desiredDict = {}
    x_lower = x_avg*disregardFactor
    x_upper = (x_avg*upperFactor)/trainProportion
    numLst = [i/2 for i in range(len(sizeDict)+1)]
    
    for j in numLst:
        try:
            # x = number of contours per bin
            x = sizeDict[j]

            if x < lower_limit:
                desiredDict[j] = 0
            elif x > upper_limit:
                desiredDict[j] = 0  
            elif x < x_lower:
                desiredDict[j] = 0
            
            elif x_lower < x < x_upper:
                desiredDict[j] = int(trainProportion*x)   
            elif x > x_upper:
                desiredDict[j] = int(x_avg*upperFactor)
                
        except KeyError:
            pass
    return desiredDict

desiredDict = obtainDesiredDict(sizeDict, disregardFactor= disregardFactor, trainProportion= trainProportion, upperFactor= upperFactor)
# print(desiredDict)

def data_standardisation(input_folder, output_folder_train, output_folder_test_val, desiredDict, input_num, bin_width, output_folder_test_val_disc):

   last_time_section = ((input_num-1)/(input_num))*200
   os.makedirs(output_folder_train, exist_ok=True)
   os.makedirs(output_folder_test_val, exist_ok=True)
   os.makedirs(output_folder_test_val_disc, exist_ok=True)
   
   
   # create the counter dictionary
   numLst = [i/2 for i in range(len(desiredDict))]
   counterDict = {}
   for j in numLst:
      counterDict[j] = 0
      

   for filename in os.listdir(input_folder):
      try:
         size = float(filename[-17:-5])
         t_number = int(filename.split("t-")[1].split("_")[0])
         if t_number > last_time_section:
            for j in numLst:
               # print(j)
               # print(size)
               if j < size < j+bin_width:
                  # print(counterDict[j])
                  # print(desiredDict[j])
                  if counterDict[j] < desiredDict[j]:
                     input_filepath = os.path.join(input_folder, filename)
                     output_filepath = os.path.join(output_folder_train, filename)
                     shutil.copy2(input_filepath, output_filepath)
                     counterDict[j] += 1
                     # print(counterDict[j])
                     for j in range(input_num-1):
                           fig_number = filename.split("_")[1]
                           t_number_new = str(int(t_number-(j+1)*(200/input_num)))
                           filename_new = [item for item in os.listdir(input_folder) if item.startswith(f"Fig_{fig_number}__t-{t_number_new}_")][0]
                           input_filepath = os.path.join(input_folder, filename_new)
                           output_filepath = os.path.join(output_folder_train, filename_new)
                           shutil.copy2(input_filepath, output_filepath)
      except (ValueError, IndexError):
         pass
   for filename2 in os.listdir(input_folder):
      if filename2 not in os.listdir(output_folder_train):
         try:
            size = float(filename2[-17:-5])
            t_number = int(filename2.split("t-")[1].split("_")[0])
            if t_number > last_time_section:
               for j in numLst:
                  if j < size < j+bin_width:
                     if desiredDict[j] < 1:
                        input_filepath = os.path.join(input_folder, filename2)
                        output_filepath = os.path.join(output_folder_test_val_disc, filename2)
                        shutil.copy2(input_filepath, output_filepath)
                        for j in range(input_num-1):
                           fig_number = filename2.split("_")[1]
                           t_number_new = str(int(t_number-(j+1)*(200/input_num)))
                           filename_new = [item for item in os.listdir(input_folder) if item.startswith(f"Fig_{fig_number}__t-{t_number_new}_")][0]
                           input_filepath = os.path.join(input_folder, filename_new)
                           output_filepath = os.path.join(output_folder_test_val_disc, filename_new)
                           shutil.copy2(input_filepath, output_filepath)
                     else:
                        input_filepath = os.path.join(input_folder, filename2)
                        output_filepath = os.path.join(output_folder_test_val, filename2)
                        shutil.copy2(input_filepath, output_filepath)
                        for j in range(input_num-1):
                           fig_number = filename2.split("_")[1]
                           t_number_new = str(int(t_number-(j+1)*(200/input_num)))
                           filename_new = [item for item in os.listdir(input_folder) if item.startswith(f"Fig_{fig_number}__t-{t_number_new}_")][0]
                           input_filepath = os.path.join(input_folder, filename_new)
                           output_filepath = os.path.join(output_folder_test_val, filename_new)
                           shutil.copy2(input_filepath, output_filepath)
         except:
            pass

   # print(counterDict)
   return counterDict
      
               

data_standardisation(input_folder=input_folder, output_folder_train=output_folder_train, output_folder_test_val=output_folder_test_val, desiredDict=desiredDict, input_num=input_num, bin_width= bin_width) 